In [3]:
ls ../node2vec/

LICENSE.md  README.md  emb/  graph/  node2vec_spark/  requirements.txt  src/


In [10]:
!pip2 install networkx
!pip2 install gensim

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
     |████████████████████████████████| 24.2MB 16.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/37/c0/25d19badc495428dec6a4bf7782de617ee0246a9211af75b302a2681dea7/smart_open-1.8.4.tar.gz
     |████████████████████████████████| 17.0MB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 133kB 17.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/23/10/c0b78c27298029e4454a472a1919bde20cb182dab1662cec7f2ca1dcc523/boto-2.49.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd3

In [70]:
!python ../node2vec/src/main.py --input ../node2vec/graph/karate.edgelist --output ../node2vec/emb/karate.emd

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [13]:
from multiprocessing.dummy import Pool # use threads
from subprocess import check_output

In [14]:
from subprocess import Popen, PIPE
import glob


In [22]:
f_list = glob.glob('../data/graph2vec_input/*.txt')



In [44]:
def outputlist(input_filename):
    temp = input_filename.split('/')
    temp[2] = 'graph2vec_output'
    return "/".join(temp)

In [47]:
output_filename = [outputlist(f_list[i]) for i in range(len(f_list))]

In [48]:
len(output_filename)

21502

In [51]:
for i,j in [f_list,output_filename]:
    print(i,j)

ValueError: too many values to unpack

In [56]:
import numpy as np
x = [[1, 2],
     [3, 4]]
np.concatenate([f_list, output_filename])

array(['../data/graph2vec_input/-68026.txt',
       '../data/graph2vec_input/67295.txt',
       '../data/graph2vec_input/-233971.txt', ...,
       '../data/graph2vec_output/66192.txt',
       '../data/graph2vec_output/231984.txt',
       '../data/graph2vec_output/-233968.txt'], dtype='|S36')

In [62]:
file_name = [[f_list[i],output_filename[i]] for i in range(len(f_list))]

In [68]:
'../data' + "/graph2vec_output/"

'../data/graph2vec_output/'

In [69]:
!mkdir ../data/graph2vec_output/

In [ ]:
cmds_list = [['python', '../node2vec/src/main.py','--input',input_filename, '--output', output_filename] for input_filename, output_filename in file_name]
procs_list = [Popen(cmd, stdout=PIPE, stderr=PIPE) for cmd in cmds_list]
for proc in procs_list:
    proc.wait()

In [ ]:
def md5sum(filename):
    try:
        return check_output(["md5sum", filename]), None
    except Exception as e:
        return None, e

if __name__ == "__main__":
    p = Pool(number_of_processes) # specify number of concurrent processes
    with open("md5sums.txt", "wb") as logfile:
        for output, error in p.imap(md5sum, filenames): # provide filenames
            if error is None:
               logfile.write(output)

In [ ]:
def read_graph():
    '''
    Reads the input network in networkx.
    '''
    if args.weighted:
        G = nx.read_edgelist(args.input, nodetype=int, data=(('weight',float),), create_using=nx.DiGraph())
    else:
        G = nx.read_edgelist(args.input, nodetype=int, create_using=nx.DiGraph())
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight'] = 1

    if not args.directed:
        G = G.to_undirected()

    return G

def learn_embeddings(walks):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    '''
    walks = [map(str, walk) for walk in walks]
    model = Word2Vec(walks, size=128, window=args.window_size, min_count=0, sg=1, workers=args.workers, iter=args.iter)
    model.save_word2vec_format(args.output)

    return

In [ ]:
nx_G = read_graph()
G = node2vec.Graph(nx_G, True, 1, 1)
G.preprocess_transition_probs()
walks = G.simulate_walks(args.num_walks, args.walk_length)
learn_embeddings(walks)